In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

In [ ]:
import GA
import TS
import MA

In [ ]:
def genertateWH(N=150, minv=-5000, maxv=5000, minlc=25, maxlc=28, unitCost=2000):
    """生成仓库坐标 N-仓库数 仓库坐标范围[minv,maxv] 仓库每平方米租赁费用[minlc,maxlc] unitCost-每个仓库面积/平方米"""
    warehouses = np.random.randint(low=minv, high=maxv+1, size=(N, 2))
    leaseCosts = np.around(np.random.randint(low=minlc, high=maxlc, size=N) + np.random.rand(N)) * unitCost
    return warehouses, leaseCosts

In [ ]:
def iterateVSvalue(namelist,tlist,epoch):
    n = len(namelist)
    plt.title("Number of iterations VS. Cost")
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    for i in range(n):
        plt.plot(list(range(1,epoch+1)),tlist[i],label=namelist[i])
    plt.legend()
    fileName = "%s%s%s%s" %("收敛过程-","迭代次数",str(epoch),".png")
    plt.savefig(fileName)
    plt.show()
    return

In [ ]:
def lab1(warehouses, leaseCosts, Q=10, Y=100, n1=100, n2=20, nxbit=3, epoch=1000):
    ts = TS.TS_4_WHouseAlloc(warehouses,leaseCosts, Q, Y, n=n1, nxbit=nxbit, max_epoch=epoch)
    v1,s1,t1 = ts.run()
    ga = GA.GA_4_WHouseAlloc(warehouses, leaseCosts, Q, Y, matingRate=0.8, variationRate=0.01, max_epoch=epoch)
    v2,s2,t2 = ga.run()
    ma = MA.MA_4_WHouseAlloc(warehouses, leaseCosts, Q, Y, matingRate=0.8, variationRate=0.01, n=n2, nxbit=nxbit, max_epoch=epoch)
    v3,s3,t3 = ma.run()
    iterateVSvalue(['TS','GA','MA'],[t1,t2,t3],epoch)
    s = [s1,s2,s3]
    var_arr=list()
    for i in s:
        wh = np.nonzero(i)[0]  #仓库号
        dist = list()
        for j in wh:
            dist.append(np.sqrt(np.square(warehouses[j][0]) + np.square(warehouses[j][1])))
        dist.append(np.var(dist))
        var_arr.append(dist)
    var_arr = np.around(np.array(var_arr).T, decimals=3)
    var_arr = pd.DataFrame(var_arr)
    var_arr.columns = ['TS算法','GA算法','Memetic算法']
    var_arr.to_excel('云仓储距离商家的距离.xlsx',index=False)
    return

In [ ]:
def lab2(Q=10, Y=100, n1=100, n2=20, nxbit=3, epoch=1000):
    v_ts = list()   # TS的最优值
    var_ts = list() # TS的方差
    v_ga = list()   # GA的最优值
    var_ga = list() # GA的方差
    v_ma = list()   # MA的最优值
    var_ma = list() # MA的方差
    x_list = list(range(50,501,50))
    for n in x_list:
        warehouses, leaseCosts = genertateWH(N = n)  #生成数据集
        '''TS算法'''
        ts = TS.TS_4_WHouseAlloc(warehouses,leaseCosts, Q, Y, n=n1, nxbit=nxbit, max_epoch=epoch)
        v1,s1,t1 = ts.run()
        v_ts.append(v1)
        wh = np.nonzero(s1)[0]  #仓库号
        dist = list()
        for j in wh:
            dist.append(np.sqrt(np.square(warehouses[j][0]) + np.square(warehouses[j][1])))
        var_ts.append(np.var(dist))
        '''GA算法'''
        ga = GA.GA_4_WHouseAlloc(warehouses, leaseCosts, Q, Y, matingRate=0.8, variationRate=0.01, max_epoch=epoch)
        v2,s2,t2 = ga.run()
        v_ga.append(v2)
        wh = np.nonzero(s2)[0]  #仓库号
        dist = list()
        for j in wh:
            dist.append(np.sqrt(np.square(warehouses[j][0]) + np.square(warehouses[j][1])))
        var_ga.append(np.var(dist))
        '''Memetic算法'''
        ma = MA.MA_4_WHouseAlloc(warehouses, leaseCosts, Q, Y, matingRate=0.8, variationRate=0.01, n=n2, nxbit=nxbit, max_epoch=epoch)
        v3,s3,t3 = ma.run()
        v_ma.append(v3)
        wh = np.nonzero(s3)[0]  #仓库号
        dist = list()
        for j in wh:
            dist.append(np.sqrt(np.square(warehouses[j][0]) + np.square(warehouses[j][1])))
        var_ma.append(np.var(dist))
    '''pic1'''
    plt.title("Number of WareHouse VS. Cost")
    plt.xlabel('Number of WareHouse')
    plt.ylabel('Cost')
    plt.plot(x_list,v_ts,label='TS')
    plt.plot(x_list,v_ga,label='GA')
    plt.plot(x_list,v_ma,label='MA')
    plt.legend()
    fileName = "%s%s%s%s" %("基于三种算法的云仓储物流成本总费用多实例对比图","-迭代次数",str(epoch),".png")
    plt.savefig(fileName)
    plt.show()  
    '''pic2'''
    plt.title("Number of WareHouse VS. Distance Var")
    plt.xlabel('Number of WareHouse')
    plt.ylabel('Distance Var')
    plt.plot(x_list,var_ts,label='TS')
    plt.plot(x_list,var_ga,label='GA')
    plt.plot(x_list,var_ma,label='MA')
    plt.legend()
    fileName = "%s%s%s%s" %("基于三种算法的云仓储距离方差多实例对比图","-迭代次数",str(epoch),".png")
    plt.savefig(fileName)
    plt.show()   
    return

In [ ]:
if __name__ == '__main__':
    warehouses, leaseCosts = genertateWH()
    lab1(warehouses,leaseCosts)
    lab2()